Importing Liberties  

In [ ]:
import yfinance as yf
import pandas as pd  
import requests                                                                     # Importing packages from my Python venv 
from bs4 import BeautifulSoup 
import sqlite3
import locale
import os

Data Scraping 

In [ ]:
sp500_url = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'              # Assigning link to 'sp500_url'
response = requests.get(sp500_url)                                                  # Using request to get access to 'sp500_url'

In [ ]:
if response.status_code ==200:
    print('Request successful')
else:                                                                              # Checking status = 200 code using 'if' & 'else'
    print('Request not successful')

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find ('table')                                       # Using Beautifulsoup() to find table within my url in 'response'

In [ ]:
data_table = pd.read_html(str(table))[0]            # Using pandas to read link from table and assigning a value to a variable 'data_table'

In [ ]:
print("Resulting DataFrame:")

print(data_table.head(503))     

Data Formatting .tolist()

In [ ]:
tickers = data_table ['Symbol'].tolist()                                                              # Assigning 'tickers' to only the 'Symbol' of the 'data_table' and convert into a list()
tickers_list = [ticker.replace('BF.B', 'BF-B').replace('BRK.B', 'BRK-B') for ticker in tickers]       # Replacing two incorrect Value 'tickers' within a variable 'tickers_list'

print('Tickers list')
print(tickers_list) 

In [ ]:
sectors_list = data_table ['GICS Sector'].tolist()       # Convert 'GICS Sector' into a list() within 'data_table' assigned to 'sectors_list'

print('Sectors list')
print(sectors_list)

In [ ]:
company_list = data_table ['Security'].tolist()         # Convert 'Security' into a list() within 'data_table' assigned to 'stocks_list'

print('Stocks list')
print(company_list)

In [ ]:
Industry_list = data_table ['GICS Sub-Industry'].tolist()                     # Convert 'GICS Sub-Industry' into a list() within 'data_table' assigned to 'Industry_list'

print('Industry list')
print(Industry_list)

In [ ]:
Location_list = data_table ['Headquarters Location'].tolist()                                  # Convert 'Headquarters Location' into a list() within 'data_table' assigned to 'Location_list'

print('Headquarters Location list')
print(Location_list)

DataFrame Creation

In [ ]:

Sp500_columns = pd.DataFrame({
    'Ticker': tickers_list,
    'Company': company_list,                                                  # Adding my variables ('tickers_list','stocks_list, and 'sectors_list') to a new Dataframe 'Sp500_columns' using pandas & {}
    'Sector': sectors_list,
    'Industry': Industry_list,
    'Location': Location_list,}) 

print(Sp500_columns)

Importing Data From Library

In [ ]:
tickers_volume = yf.download(tickers_list, period="1y", interval="1d")[['Volume']]    # Using 'yf' to download all the info in the 'tickers_list' between those dates and assigning it to 'tickers_price'

Calculate The Volume For Each Ticker

In [ ]:
daily_volume = tickers_volume['Volume'].resample('D').sum()                                                # Calculate the daily volume for each ticker

monthly_volume = tickers_volume['Volume'].resample('M').mean()                                           # Calculate the monthly average volume for each ticker

weekly_average_volume = tickers_volume['Volume'].resample('W').mean()                               # Calculate the daily average volume for each ticker

yearly_average_volume = tickers_volume['Volume'].resample('Y').mean()                               # Calculate the yearly average volume for each ticker

Importing Market Caps From Each Ticker 

In [ ]:
data = []

for ticker in tickers_list:
    try:                                             # Importing Tickers from S&P 500 (thicker_list)              
        stock = yf.Ticker(ticker)
        info = stock.info
        
        
                                                     # Extracting information
        market_cap = info.get('marketCap', 'N/A')
        name =  'N/A'  
        sector = 'N/A'  
        
                                                     # Append the data to the list
        data.append({
            'Ticker': ticker,
            'Company': name,
            'Sector': sector,
            'Market_Cap': market_cap
        })

    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")

                                                    # Create a DataFrame from the list of data
SP500 = pd.DataFrame(data)

SP500_MCS = SP500.sort_values(by='Market_Cap', ascending=False)


In [ ]:
SP500_MCS.drop('Company', axis=1, inplace=True)
                                                                     # Dropping Unnecessary columns from DataFrame(SP500_MC)
SP500_MCS.drop('Sector', axis=1, inplace=True)

print(SP500_MCS)

Data Formatting (Market Cap) & Function Creation 

In [ ]:
def format_market_cap(market_cap):
    if isinstance(market_cap, (int, float)):
                                                                   # If it's already a numeric value, format it accordingly
        if market_cap >= 1e12:
            return "${:.2f} T".format(market_cap / 1e12)
        elif market_cap >= 1e9:
            return "${:.2f} B".format(market_cap / 1e9)
        elif market_cap >= 1e6:
            return "${:.2f} M".format(market_cap / 1e6)
        else:
            return "${:.2f}".format(market_cap)
    else:
        try:
                                                                   # Convert the market cap value to a floating-point number
            market_cap = float(market_cap.replace('$', '').replace('B', 'e9').replace('M', 'e6').replace('T', 'e12'))

                                                                   # Check the magnitude of the market cap and append 'T', 'B', or 'M' accordingly
            if market_cap >= 1e12:
                return "${:.2f} T".format(market_cap / 1e12)
            elif market_cap >= 1e9:
                return "${:.2f} B".format(market_cap / 1e9)
            elif market_cap >= 1e6:
                return "${:.2f} M".format(market_cap / 1e6)
            else:
                return "${:.2f}".format(market_cap)
        except Exception as e:
            
            print(f"Error formatting market cap: {e}")
            return market_cap                                       # Return the original value if there's an issue with formatting


SP500_MCS['Market_Cap'] = SP500_MCS['Market_Cap'].apply(format_market_cap)

print(SP500_MCS)


Data Merging (JOIN) Drop And Inserting Column

In [ ]:
Updated_Sp500_columns = pd.merge(Sp500_columns, SP500_MCS[['Ticker', 'Market_Cap']], left_on='Ticker', right_on='Ticker', how='right')             # pd.merge is used to merge two DataFrames 'JOIN'

                                                                                   
Updated_Sp500_columns.drop('Industry', axis=1, inplace=True)                                                                                        # Dropping Unnecessary Columns in merged DataFrames


Updated_Sp500_columns.insert(0, 'Rank', range(1, len(Updated_Sp500_columns) + 1))

 Function Creation (Date, Open, & Close)

In [ ]:
def calculate_Date_weekly(ticker):
    OC_Date = yf.download(ticker, period="1y", interval="1d")
    
                                                             # Check if the index is a DatetimeIndex
    if not isinstance(OC_Date.index, pd.DatetimeIndex):
        raise KeyError(f"DatetimeIndex not found in {ticker} data.")

                                                            # Resample the data to weekly frequency, considering Friday as the end of the week
    Weekly_Date = OC_Date.resample('W-Fri').last()  # Use 'last' to get the last data point for the week
    
    
                                                             # Get the most recent weekly date value
    most_recent_weekly_date = Weekly_Date.index[-1]
    
                                                             # Extract the date part from the string
    date_part = most_recent_weekly_date.strftime('%Y-%m-%d')

    return date_part

Updated_Sp500_columns['Date'] = Updated_Sp500_columns['Ticker'].apply(calculate_Date_weekly)

In [ ]:

def calculate_open_weekly_average(ticker):                                                                               # Defines a function to calculate the Open
    Open_Close = yf.download(ticker , period= "1y", interval="1d")
    Open_SP500 = Open_Close['Open'].resample('W').mean()                                                                 # Calculates the mean of the Open for each ticker
                                                                                                                         
    most_recent_weekly_open = Open_SP500.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_weekly_open 

Updated_Sp500_columns['Weekly_Open'] = Updated_Sp500_columns['Ticker'].apply(calculate_open_weekly_average)

In [ ]:
def calculate_close_weekly_average(ticker):                                                                               # Defines a function to calculate the Close
    Open_Close = yf.download(ticker, period= "1y", interval="1d")
    Close_SP500 = Open_Close['Close'].resample('W').mean()                                                # Calculates the mean of the Close for each ticker
    
    most_recent_weekly_close = Close_SP500.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_weekly_close  

Updated_Sp500_columns['Weekly_Close'] = Updated_Sp500_columns['Ticker'].apply(calculate_close_weekly_average)

Column Creation Weekly Percentage (Open/Close) 

In [ ]:
# Calculate the percentage change and add the column to the DataFrame
Updated_Sp500_columns['Weekly_Percentage'] = (((Updated_Sp500_columns['Weekly_Close'] - Updated_Sp500_columns['Weekly_Open']) / Updated_Sp500_columns['Weekly_Open']) * 100)

 Function Creation (Weekly Volume & Monthly Volume)

In [ ]:
def calculate_weekly_average(ticker):                                                                               # Defines a function to calculate the weekly average
    ticker_data = yf.download(ticker, period="1y", interval="1d")
    weekly_average = ticker_data['Volume'].resample('W').mean()                                                     # Calculates the mean of the weekly average volumes for each ticker
    
    most_recent_weekly_volume = weekly_average.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_weekly_volume  
  

Updated_Sp500_columns['Weekly_Volume'] = Updated_Sp500_columns['Ticker'].apply(calculate_weekly_average)

In [ ]:
def calculate_monthly_average(ticker):                                                                               # Defines a function to calculate the Daily Volume
    ticker_monthly_data = yf.download(ticker, period="1y", interval="1d")
    monthly_average = ticker_monthly_data['Volume'].resample('M').mean()                                                # Calculates the mean of the Daily Volume for each ticker
    
    most_recent_monthly_volume = monthly_average.iloc[-1]                                                                        # Get the most recent weekly open value
    return most_recent_monthly_volume    

Updated_Sp500_columns['Monthly_Volume'] = Updated_Sp500_columns['Ticker'].apply(calculate_monthly_average)

Data Formatting (Volume & Open/Close) 

In [ ]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Average Weekly Open' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Weekly_Open'] = Updated_Sp500_columns['Weekly_Open'].apply(lambda x: locale.format_string("%d", x, grouping=True))

In [ ]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Average Weekly Close' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Weekly_Close'] = Updated_Sp500_columns['Weekly_Close'].apply(lambda x: locale.format_string("%d", x, grouping=True))

In [ ]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Weekly Average Volume' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Weekly_Volume'] = Updated_Sp500_columns['Weekly_Volume'].apply(lambda x: locale.format_string("%d", x, grouping=True))

In [ ]:
locale.setlocale(locale.LC_ALL, '')                                                                                # Using 'locale' to  formatting the 'Daily Volume' column in a DataFrame by adding commas as thousand separators to make it more readable.

Updated_Sp500_columns['Monthly_Volume'] = Updated_Sp500_columns['Monthly_Volume'].apply(lambda x: locale.format_string("%d", x, grouping=True))

Changing Data Types

In [ ]:
Updated_Sp500_columns['Weekly_Open'] = Updated_Sp500_columns['Weekly_Open'].astype(int)                   

Updated_Sp500_columns['Weekly_Close'] = Updated_Sp500_columns['Weekly_Close'].astype(int)                    # Converting Columns To int()
 
Updated_Sp500_columns['Weekly_Volume'] = Updated_Sp500_columns['Weekly_Volume'].astype(int)                   

Updated_Sp500_columns['Monthly_Volume'] = Updated_Sp500_columns['Monthly_Volume'].astype(int)               

Reapplying Column (Market Cap) to the end 

In [ ]:
Market_Cap = Updated_Sp500_columns.pop('Market_Cap')
Updated_Sp500_columns['Market_Cap'] = Market_Cap

print("Resulting S&P 500 DataFrame:")

Updated_Sp500_columns

Exporting DataFrame to Tableau 

In [ ]:
# Tableau file path
Tableau_path = '/Users/Phases/Desktop/Project XV/Tableau/Tableau_XV.csv'

# Save the DataFrame to the CSV file, overwriting if it already exists
Updated_Sp500_columns.to_csv(Tableau_path, index=False)

Applying Percentage Sign %

In [ ]:
Updated_Sp500_columns['Weekly_Percentage'] = (Updated_Sp500_columns['Weekly_Percentage'].astype(float).round(2).astype(str) + '%')

Checking Data Types

In [ ]:
Updated_Sp500_columns.info()                              # Calling All The Info() Of This DataFrame 

Exporting DataFrame To SQLite With Primary Key (Recreation Of Tables)  

In [ ]:
# New database file path
project_path = '/Users/Phases/Desktop/Project XV'
instance_folder = 'Instance'
SP_Database = 'SP500_Database.db'
SP500_Database_Path = os.path.join(project_path, instance_folder, SP_Database)

# Connect to the SQLite database using the new file path
conn = sqlite3.connect(SP500_Database_Path)

Updated_Sp500_columns.to_sql('SP500_Columns_Key', conn, index=False, if_exists='replace')   # Adding DataFrame To SQLite Database

cursor = conn.cursor()    # Starting SQL Query 

                          # Creating New Table With Primary Key
cursor.execute('''         
    CREATE TABLE IF NOT EXISTS SP500_Columns_Key_New (
        id INTEGER PRIMARY KEY,
        Rank INTEGER,
        Ticker TEXT,
        Company TEXT,
        Sector TEXT,
        Location TEXT,
        Date DATE,
        Weekly_Open INTEGER,
        Weekly_Close INTEGER,
        Weekly_Percentage TEXT,
        Weekly_Volume INTEGER,
        Monthly_Volume INTEGER,
        Market_Cap TEXT
    )''')


conn.commit()

                          # Copying Data From Existing Table To The New Table With Primary Key
cursor.execute('''
    INSERT INTO SP500_Columns_Key_New (Rank, Ticker, Company, Sector, Location, Date, Weekly_Open, Weekly_Close, Weekly_Percentage, Weekly_Volume, Monthly_Volume, Market_Cap)    
    SELECT Rank, Ticker, Company, Sector, Location, Date, Weekly_Open, Weekly_Close, Weekly_Percentage, Weekly_Volume, Monthly_Volume, Market_Cap
    FROM SP500_Columns_Key
''')

conn.commit()

                         # Renaming Both Tables One With Primary Key and One Without
cursor.execute('''
    ALTER TABLE SP500_Columns_Key RENAME TO SP500_Columns_Key_Old
''')

cursor.execute('''
    ALTER TABLE SP500_Columns_Key_New RENAME TO SP500_Columns_Key
''')

conn.commit()

                        # Dropping Old Table 
cursor.execute('''
    DROP TABLE IF EXISTS SP500_Columns_Key_Old
''')

conn.close()

Exporting DataFrame to Excel

In [ ]:
folder_path = '/Users/Phases/Desktop/Project XV/Excel'
file_name = 'SP500_excel.xlsx'

# Get the full path to the Excel file
excel_file_path = os.path.join(folder_path, file_name)

# Make sure the folder exists, create it if not
os.makedirs(folder_path, exist_ok=True)

# Save the DataFrame to Excel
Updated_Sp500_columns.to_excel(excel_file_path, index=False, )